<a href="https://colab.research.google.com/github/tomvangasse/game-store-manager/blob/main/tomsMenuProgram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
'''
Upload files here
Click run and then click choose files:

    Video_Game_Info.txt
    Board_Game_Info.txt
    Subscription_Info.txt
    Game_feedback.txt
    Booking.txt
    Rental.txt
    feedbackManager.pyc
    subscriptionManager.pyc
'''


from google.colab import files
uploaded = files.upload()

Saving Board_Game_Info.txt to Board_Game_Info (1).txt
Saving Booking.txt to Booking (1).txt
Saving Game_Feedback.txt to Game_Feedback (1).txt
Saving Rental.txt to Rental (1).txt
Saving Subscription_Info.txt to Subscription_Info (1).txt
Saving Video_Game_Info.txt to Video_Game_Info (1).txt
Saving feedbackManager.pyc to feedbackManager (1).pyc
Saving subscriptionManager.pyc to subscriptionManager (1).pyc


Test

In [2]:
# import ipywidgets as widgets
# from ipywidgets import Layout
# from IPython.display import clear_output


# dateWidget = widgets.DatePicker(
#     description='Pick a Date',
#     disabled=False
# )

# textbox = widgets.Text(
#     value='',
#     placeholder='Type something',
#     description='String:',
#     disabled=False
# )

# button = widgets.Button(description="Submit")
# output = widgets.Output()



# def on_button_clicked(b):
#     with output:
#         clear_output()
#         button.layout.visibility = 'hidden'
#         print(dateWidget.value)
#         print(textbox.value)

# button.on_click(on_button_clicked)
# display(dateWidget)
# display(textbox)
# display(button, output)


Register / login

In [3]:
def getUserID():
    while True:
        userID = input("Enter user ID (or q to quit): ")
        if userID == "q":
            return False
        if checkUserLength(userID):
            userID = checkSubscription(userID)
            return userID
        else:
            print("Invalid user ID")


def checkSubscription(userID):
    while True:
        subscriptions = smTV.load_subscriptions()
        if smTV.check_subscription(userID, subscriptions):
            return userID
        else:
            login(userID)

    subType = subscriptions[userID]["SubscriptionType"]


def register(userID):
    import subscriptionManager as smTV
    import datetime
    from dateutil.relativedelta import relativedelta as rd

    print("Unregistered account, please register now:")
    while True:
        subscriptionType = input("Enter subscription type (Basic or premium): ").lower()
        if subscriptionType != "basic" and subscriptionType != "premium":
            print("Invalid subscription type")
        else:
            break
    date = datetime.datetime.now()
    startTime = date.strftime("%Y-%m-%d")
    endTime = (date + rd(years=3)).strftime("%Y-%m-%d")

    with open("Subscription_Info.txt", mode = "a") as file:
        file.write(f"\n{userID},{subscriptionType},{startTime},{endTime}")

    print(f"Logged in as {userID}")
    return subscriptionType


def login(userID):
    import subscriptionManager as smTV

    subscriptions = smTV.load_subscriptions()

    if smTV.check_subscription(userID, subscriptions):
        print(f"Logged in as {userID}")

    else:
        register(userID)

def checkUserLength(userID):
    if len(userID) != 4:
        return False
    else:
        return True

Game search

In [47]:
import csv


output = widgets.Output()


def getVideoDict():
    with open("Video_Game_Info.txt", mode="r", newline="") as file:
        reader = csv.DictReader(file)
        boardGameDict = {row["GameID"]: row for row in reader}
    return boardGameDict

def getBoardDict():
    with open("Board_Game_Info.txt", mode="r", newline="") as file:
        reader = csv.DictReader(file)
        boardGameDict = {row["GameID"]: row for row in reader}
    return boardGameDict

def printGames(platform, genre):
    videoDict = getVideoDict()
    boardDict = getBoardDict()
    with output:
        clear_output()
        for i in boardDict:
            currentGame = boardDict[i]
            if platform == "All" and genre in boardDict[i]["Genre"]:
                print(f"{currentGame["GameID"]}: {(currentGame["Game name"]): <25} Available / Unavailable: {currentGame["Available copies"]} / {currentGame["Unavailable copies"]}")
        for i in videoDict:
            currentGame = videoDict[i]
            if platform in currentGame["Platform"] and genre in videoDict[i]["Genre"]:
                print(f"{currentGame["GameID"]}: {(currentGame["Game name"]): <25} Available / Unavailable: {currentGame["Available copies"]} / {currentGame["Unavailable copies"]}")
            elif platform == "All" and genre in currentGame["Genre"]:
                print(f"{currentGame["GameID"]}: {(currentGame["Game name"]): <25} Available / Unavailable: {currentGame["Available copies"]} / {currentGame["Unavailable copies"]}")
            elif platform in currentGame["Platform"] and genre == "All":
                print(f"{currentGame["GameID"]}: {(currentGame["Game name"]): <25} Available / Unavailable: {currentGame["Available copies"]} / {currentGame["Unavailable copies"]}")



def printGameInfo(GameID):
    videoDict = getVideoDict()
    boardDict = getBoardDict()
    if GameID == "":
        return
    with output:
        clear_output()
        if GameID in boardDict:
            gameInfo = boardDict[GameID]
            for key, value in gameInfo.items():
                print(f"{key: >20}:  {value}")
            print("\n")
        elif GameID in videoDict:
            gameInfo = videoDict[GameID]
            for key, value in gameInfo.items():
                print(f"{key: >20}:  {value}")
            print("\n")
        else:
            return



def gameSearch():
    videoDict = getVideoDict()
    boardDict = getBoardDict()
    with output:
        gameType = input('''Board games (b)
Video games (v)
Quit (q)
''')

    if gameType == "q":
        return False

    with output:
        gameSelection = input('''Print games (press enter)
Select a specific game (enter GameID)
''')

    gameSelection
    if gameSelection == "":
        printGames(gameType)
    elif gameSelection == "q":
        return False
    else:
        printGameInfo(gameSelection, gameType)



Update text files:

In [8]:
def updateVideo(videoDict):
    headers = ["GameID", "Game name", "Platform", "Genre", "Date added", "Available copies", "Unavailable copies"]
    writer = csv.DictWriter(open("Video_Game_Info.txt", mode="w", newline=""), fieldnames=headers)
    writer.writeheader()
    for key, value in videoDict.items():
        writer.writerow(value)
    return

def updateBoard(boardDict):
    headers = ["GameID", "Game name", "No of players", "Genre", "Date added", "Available copies", "Unavailable copies"]
    writer = csv.DictWriter(open("Board_Game_Info.txt", mode="w", newline=""), fieldnames=headers)
    writer.writeheader()
    for key, value in boardDict.items():
        writer.writerow(value)
    return

def updateRental(rentalList):
    with open("Rental.txt", mode="w", newline="") as file:
        file.write("Rental userID,GameID,Rental date,Return date\n")
        for row in rentalList:
            file.write(f"{','.join(row)}\n")
    return



Game rent

In [9]:
import subscriptionManager as smTV
import feedbackManager as fbTV


def updateBoardFromRentals(GameID, action):
    value = 0
    if action == "rent":
        value = 1
    elif action == "return":
        value = -1
    else:
        return

    boardDict = getBoardDict()
    availableCopies = boardDict[GameID]["Available copies"]
    unavailableCopies = boardDict[GameID]["Unavailable copies"]
    boardDict[GameID]["Available copies"] = str(int(availableCopies) - value)
    boardDict[GameID]["Unavailable copies"] = str(int(unavailableCopies) + value)
    updateBoard(boardDict)

def updateVideoFromRentals(GameID, action):
    value = 0
    if action == "rent":
        value = 1
    elif action == "return":
        value = -1
    else:
        return

    videoDict = getVideoDict()
    availableCopies = videoDict[GameID]["Available copies"]
    unavailableCopies = videoDict[GameID]["Unavailable copies"]
    videoDict[GameID]["Available copies"] = str(int(availableCopies) - value)
    videoDict[GameID]["Unavailable copies"] = str(int(unavailableCopies) + value)
    updateVideo(videoDict)

def updateRentGame(gameID, userID):
    from dateutil.relativedelta import relativedelta as rd
    import datetime

    date = datetime.datetime.now()
    startTime = date.strftime("%Y-%m-%d")
    with open("Rental.txt", mode="a", newline="") as file:
        file.write(f"\n{userID},{gameID},{startTime},")


def checkAvailability(gameID, gameType):
    videoDict = getVideoDict()
    boardDict = getBoardDict()
    if gameType == "b":
        currentGame = boardDict[gameID]
        if currentGame["Available copies"] == "0":
            return False
        else:
            return True

    elif gameType == "v":
        currentGame = videoDict[gameID]
        if currentGame["Available copies"] == "0":
            return False
        else:
            return True
    else:
        return


def rentGameMain():
    videoDict = getVideoDict()
    boardDict = getBoardDict()
    userID = getUserID()
    if userID == False:
        return

    selectedGameID = input('''Enter the GameID for the desired game:
''')
    if selectedGameID in videoDict:
        if checkAvailability(selectedGameID, "v"):
            updateVideoFromRentals(selectedGameID, "rent")
            updateRentGame(selectedGameID, userID)
            currentGame = videoDict[selectedGameID]
            print(f"{currentGame["Game name"]} has been rented by {userID}")

        else:
            print("Game not available")

    elif selectedGameID in boardDict:
        # print(checkAvailability(selectedGameID, "b"))
        if checkAvailability(selectedGameID, "b"):
            updateBoardFromRentals(selectedGameID, "rent")
            updateRentGame(selectedGameID, userID)
            currentGame = boardDict[selectedGameID]
            print(f"{currentGame["Game name"]} has been rented by {userID}")

        else:
            print("Game not available")

    else:
        print("Invalid GameID")



#with open("Rental.txt", mode="a") as file:
    #file.write(f"\n{gameID},{Rental date}

rentGameMain()

Enter user ID (or q to quit): q


Returns


In [10]:
import datetime


def getRentalList():
    with open("Rental.txt", mode="r", newline="") as file:
        count = 0
        rentalList = []
        for line in file:
            if count == 0:
                count += 1
                continue
            else:
                currentLine = line.rstrip().split(",")
            count += 1
            rentalList.append(currentLine)
    return rentalList

def getUserRental(userID, rentalList):
    userRentals = []
    for row in rentalList:
        if row[0] == userID and row[3] == "":
            userRentals.append(row)
    return userRentals

def printRentedGames(userRentals):
    print("\nRented games:")
    videoDict = getVideoDict()
    boardDict = getBoardDict()
    for row in userRentals:
        gameID = row[1]
        if gameID in videoDict:
            print(f"{gameID}:  {videoDict[gameID]['Game name']}")
        elif gameID in boardDict:
            print(f"{gameID}:  {boardDict[gameID]['Game name']}")
        else:
            print("Invalid GameID")

def updateReturnList(rentalList, userID, gameID):
    for row in rentalList:
        if row[0] == userID and row[1] == gameID and row[3] == '':
            row[3] = datetime.datetime.now().strftime("%Y-%m-%d")
            break
    return rentalList


def returnGameMain():
    videoDict = getVideoDict()
    boardDict = getBoardDict()

    date = datetime.datetime.now()
    rentalList = getRentalList()
    userID = getUserID()
    if userID == False:
        return
    userRental = getUserRental(userID, rentalList)
    if len(userRental) == 0:
        print("No rented games")
        return
    printRentedGames(userRental)

    while True:
        selectedGameID = input("Enter selected gameID: ")
        for row in userRental:
            if row[1] == selectedGameID:
                break
        else:
            print("Invalid GameID")
            continue
        break


    rentalList = updateReturnList(rentalList, userID, selectedGameID)
    if selectedGameID in videoDict:
        updateVideoFromRentals(selectedGameID, "return")
    elif selectedGameID in boardDict:
        updateBoardFromRentals(selectedGameID, "return")
    updateRental(rentalList)

    print("Game returned")

returnGameMain()


Enter user ID (or q to quit): q


Booking

In [11]:
import datetime

def updateBooking(userID, date, time, guests):
    with open("Booking.txt", mode="a", newline="") as file:
        file.write(f"\n{userID},{date},{time},{guests}")
    return

def bookingMain():

    userID = getUserID()
    if userID == False:
        return

    date = input("Enter date in format YYYY-MM-DD: ")

    time = input("Enter time in format 2pm / 6pm")

    guests = input("Enter number of guests: ")

    updateBooking(userID, date, time, guests)

    print("Booking confirmed")

bookingMain()

Enter user ID (or q to quit): q


Main

In [48]:
import ipywidgets as widgets
from ipywidgets import Layout
from IPython.display import clear_output

onSearch = 0
onRent = 0
onReturn = 0
onBook = 0

def on_search_clicked(b):
    onSearch = 1
    menuBox.layout.display = 'none'
    quitBox.layout.display = ''

    ioBox.layout.display = ''
    with output:
        clear_output()


def on_rent_clicked(b):
    onRent = 1
    menuBox.layout.display = 'none'
    quitBox.layout.display = ''
    ioBox.layout.display = ''

def on_return_clicked(b):
    onReturn = 1
    menuBox.layout.display = 'none'
    quitBox.layout.display = ''
    ioBox.layout.display = ''

def on_book_clicked(b):
    onBook = 1
    menuBox.layout.display = 'none'
    quitBox.layout.display = ''
    ioBox.layout.display = ''

def main_menu(b):
    menuBox.layout.display = ''
    quitBox.layout.display = 'none'
    ioBox.layout.display = 'none'

    onSearch = 0
    onRent = 0
    onReturn = 0
    onBook = 0

def on_search_for_game_clicked(b):
    if enterGameID.value == "":
        printGames(dropdownPlatform.value, dropdownGenre.value)
    else:
        printGameInfo(enterGameID.value)

def on_reset_click(b):
    with output:
        clear_output()


buttonSearch = widgets.Button(description="Search games")
buttonSearch.on_click(on_search_clicked)

buttonRent = widgets.Button(description="Rent a game")
buttonRent.on_click(on_rent_clicked)

buttonReturn = widgets.Button(description="Return a game")
buttonReturn.on_click(on_return_clicked)

buttonBook = widgets.Button(description="Book a timeslot")
buttonBook.on_click(on_book_clicked)

quitButton = widgets.Button(description="Quit")
quitButton.on_click(main_menu)



enterGameID = widgets.Text(placeholder="Enter GameID", layout=widgets.Layout(width='50%'))

buttonSearchForGame = widgets.Button(description="Search", layout=widgets.Layout(width='50%'))
buttonSearchForGame.on_click(on_search_for_game_clicked)

dropdownPlatform = widgets.Dropdown(options=["All", "Playstation", "Xbox", "PC", "Nintendo"],
                                    value = "All",
                                    description="Platform:",
                                    layout=widgets.Layout(width='50%'))

dropdownGenre = widgets.Dropdown(options=["All", "Action-Adventure", "RPG", "Shooter", "Simulation", "Sandbox",
                                    "Action", "Platformer", "Racing", "Strategy", "Mystery", "Family", "Quiz", "Word"],
                                    value = "All",
                                    description="Genre:")

resetButton = widgets.Button(description="Reset", layout=widgets.Layout(width='50%'))
resetButton.on_click(on_reset_click)


main_output = widgets.Output()
output = widgets.Output()

menuBox = widgets.VBox([buttonSearch, buttonRent, buttonReturn, buttonBook])
quitBox = widgets.VBox([quitButton])

quitBox.layout.display = 'none'

inputBox = widgets.HBox([enterGameID, dropdownPlatform, dropdownGenre, buttonSearchForGame, resetButton])
ioBox = widgets.VBox([inputBox, output])


mainBox = widgets.HBox([menuBox, quitBox, ioBox])

display(mainBox)
main_menu('b')


